<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item">
    <li><span><a href="#|-Preliminaries" data-toc-modified-id="|-Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>| Preliminaries</a></span></li><li><span><a href="#|-Cleaning" data-toc-modified-id="|-Cleaning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>| Cleaning</a></span></li><li><span><a href="#|-Named-Entity-Recognition-w/-spaCy" data-toc-modified-id="|-Named-Entity-Recognition-w/-spaCy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>| Named Entity Recognition w/ spaCy</a></span><ul class="toc-item"><li><span><a href="#Split-first,-middle-and-last-name-in-different-cols" data-toc-modified-id="Split-first,-middle-and-last-name-in-different-cols-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Split first, middle and last name in different cols</a></span></li><li><span><a href="#|-Enrich-first-name-if-only-last-name-is-mentioned" data-toc-modified-id="|-Enrich-first-name-if-only-last-name-is-mentioned-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>| Enrich first name if only last name is mentioned</a></span></li></ul></li><li><span><a href="#|-Gender-Guesser" data-toc-modified-id="|-Gender-Guesser-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>| Gender Guesser</a></span></li><li><span><a href="#|-Group-per-article-and-get-share-of-gender-per-article" data-toc-modified-id="|-Group-per-article-and-get-share-of-gender-per-article-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>| Group per article and get share of gender per article</a></span></li><li><span><a href="#|-Wikidata-Query" data-toc-modified-id="|-Wikidata-Query-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>| Wikidata Query</a></span></li></ul></div>

---
# | Preliminaries

In [209]:
import os
import json
import pandas as pd
import numpy as np
import regex as re
from pandarallel import pandarallel
import pathlib

#from bs4 import BeautifulSoup as bs

# SpaCy 
import spacy
import en_core_web_trf
import en_core_web_md
import en_core_web_sm

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk

# TRF Orininal / Md schneller
nlp = spacy.load("en_core_web_md")
# JULIA nlp = spacy.load("en_core_web_trf")

from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

import warnings
warnings.filterwarnings('ignore') # (action='once')

In [210]:
pandarallel.initialize(progress_bar=True)
#pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [211]:
def splitFile(TRECfile, quantity):
    """
    TREC_Washington_Post_collection.v4.jl has 728626 Lines and has 15GB.
    It is to large to read into one Dataframe for the most PCs.
    With this function you can split the big TREC-File into as many
    small files you want.
    
    args: 
    TRECfile = Complete Path + Filename
    quantitiy = How many Small files you want to create
    """
    
    # Splitting the Filename out of the Path
    TRECpath = TRECfile.split('\\')
    TRECpath = TRECfile[:(len(TRECpath[-1])*-1)-1]
    
    # Creating a subfolder 'small' for the new Files
    TRECpathSmall = os.path.join(TRECpath, 'small')
    if not os.path.exists(TRECpathSmall):
        os.makedirs(TRECpathSmall)
        print(f'{TRECpathSmall} wurde erstellt.')
    
    # If the folder 'small' is noot already there and has data skip this function
    TRECfileSmall = os.listdir(TRECpathSmall)
    if not TRECfileSmall:
        smallfiles = []
        lines_per_file = int(729000/quantity)
        
        smallfile = None
        i = 1
        try:
            with open(TRECfile) as bigfile:
                for lineno, line in enumerate(bigfile):
                    if lineno % lines_per_file == 0:
                        if smallfile:
                            smallfile.close()
                        #small_filename = 'TREC_Washington_Post_small_{}.jl'.format(lineno + lines_per_file)
                        small_filename = f'TREC_Washington_Post_small_{i}.jl'
                        small_filename = os.path.join(TRECpathSmall, small_filename)
                        print(f'{small_filename} wird erstellt')
                        smallfiles.append(small_filename)
                        i+=1
                        smallfile = open(small_filename, "w")
                    smallfile.write(line)
                if smallfile:
                    smallfile.close()
            return smallfiles
        except:
            print('TREC_Washington_Post_collection.v4.jl nicht gefunden')
    else:
        # If the folder 'small' is already there and has data create a list of the data
        TRECfileSmall = os.listdir(TRECpathSmall)
        smallfiles = []
        for smallfile in TRECfileSmall:
            smallfile = os.path.join(TRECpathSmall, smallfile)
            smallfiles.append(smallfile)
        return smallfiles

    
# PATH TO YOUR TREC_Washington_Post_collection JSON
myTRECfile = 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\TREC_Washington_Post_collection.v4.jl'
WpDataSmall = splitFile(myTRECfile, 30)

WpDataSmall

['C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\small\\TREC_Washington_Post1.pickle',
 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\small\\TREC_Washington_Post2.pickle',
 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\small\\TREC_Washington_Post3.pickle',
 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\small\\TREC_Washington_Post4.pickle']

In [212]:
def SmallFilesToPickle(listOfSmallFiles):
    fileending = listOfSmallFiles[0]
    fileending = pathlib.Path(fileending).suffix
    fileending = fileending[1:]
    if fileending == 'jl':
        listOfSmallFilesPickle = []
        for filename in listOfSmallFiles:
            df = pd.read_json(filename, lines=True)

            # Drop empty rows
            df.dropna(inplace = True)

            # delete duplicated and unnecessary columns
            if 'contents' in df.columns:
                df = df.drop(columns='contents')
            #if 'article_url' in df.columns:
            #    df = df.drop(columns='article_url')
            if 'type' in df.columns:
                df = df.drop(columns='type')
            if 'source' in df.columns:
                df = df.drop(columns='source')

            filenamemeameOld = filename
            fileNameNew = filename[:-3]+'.pickle'
            os.remove(filenamemeameOld)
            df.to_pickle(fileNameNew)
            listOfSmallFilesPickle.append(fileNameNew)
            del df
        return listOfSmallFilesPickle
    elif fileending == 'pickle':
        return listOfSmallFiles
    else:
        return None

WpDataSmall = SmallFilesToPickle(WpDataSmall)

WpDataSmall

['C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\small\\TREC_Washington_Post1.pickle',
 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\small\\TREC_Washington_Post2.pickle',
 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\small\\TREC_Washington_Post3.pickle',
 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\small\\TREC_Washington_Post4.pickle']

In [213]:
# ----------------- LÖSCHEN WENN ANDERE FILES GENUTZT WERDEN SOLLEN  -----------

hundredthousandWPpath = 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\hundredthousand_wp.pickle'

del WpDataSmall[:]
WpDataSmall.append(hundredthousandWPpath)
WpDataSmall

# -------------------------------------------------------------------------------

['C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\hundredthousand_wp.pickle']

In [214]:
def createDfFromSmallFiles(listOfSmallFiles):
    fileending = listOfSmallFiles[0]
    fileending = pathlib.Path(fileending).suffix
    fileending = fileending[1:]
    if fileending == 'pickle':
        list_of_dataframes = []
        for filename in listOfSmallFiles:
            to_merge_df = pd.read_pickle(filename)
            list_of_dataframes.append(to_merge_df)

        # write all small dataframes in one big dataframe
        df = pd.concat(list_of_dataframes)        

        # delete the small dataframes for more memory
        del to_merge_df
        del list_of_dataframes

        return df

df = createDfFromSmallFiles(WpDataSmall)

# Randomize the rows and reset a new id
#df = df.sample(frac=1).reset_index(drop=True)

# !!! DATENSATZ VERKLEINERT
df = df[:300]

---
# | Pre Processing

In [215]:
def dateconvert(x):
    try:
        import datetime  
        ts = datetime.datetime.fromtimestamp(x.published_date/1000)
        # year-month-day with hours-minutes-seconds
        #date = ts.strftime('%Y-%m-%d %H:%M:%S')
        # year-month-day without time
        date = ts.strftime('%Y-%m-%d')
        return date
    except:
        return x.published_date

# converts the epochs datetime to a normale datestring
df['published_date'] = df.parallel_apply(dateconvert, axis=1)

In [216]:
def createText(x):
    try:
        import pandas as pd
        dfcontent = pd.json_normalize(x.content, max_level=1)
        dfcontent = dfcontent[dfcontent.subtype == 'paragraph']
        #dfcontent = dfcontent[dfcontent.mime == 'text/html']
        contentText = ''
        
        for index, row in dfcontent.iterrows():
            contentText += row["content"]
            contentText += ' '
            
            import re
            contentText = contentText.encode('utf-8').decode('ascii','ignore')
            contentText = re.sub('<[^<]+?>', '', contentText)
            contentText = contentText.replace(u'\xa0', u' ')
            contentText = contentText.replace(f'\\n', ' ')
            contentText = contentText.replace(f'\n', f' ')
            contentText = contentText.replace('  ',' ')
            
        return contentText
    except:
        return None

# creates text from the content column by filtering html and json
df['text'] = df.parallel_apply(createText, axis=1)

# if content json is converted in text delete the column
# !!! df = df.drop(columns='content')

---
# | Cleaning

In [217]:
df.head(5)

,id,article_url,title,author,published_date,type,source,content,publish_date,orig-id,text
0,ABM35HDPIII6RP4GUI2RWXWOTE,/opinions/the-hollow-compassion-of-trump-wary-...,Trump’s America goes full Charles Dickens,Dana Milbank,NaN,NaN,NaN,"[{'content': 'Opinions', 'mime': 'text/plain',...",NaN,NaN,Donald Trumps America has taken on a Dickensia...
1,7f4cc4bc-351a-11e4-9e92-0899b306bbea,https://www.washingtonpost.com/local/u-md-bask...,U-Md. basketball arena is transformed into mak...,Karen Chen,2014-09-06,article,The Washington Post,"[{'content': 'Local', 'mime': 'text/plain', 't...",NaN,NaN,Outside the University of Marylands basketball...
2,aa34106e-bc85-11e1-8867-ecf6cb7935ef,https://www.washingtonpost.com/opinions/mitch-...,Mitch McConnell: How political disclosure coul...,Mitch McConnell,2012-06-23,article,The Washington Post,"[{'content': 'Opinions', 'mime': 'text/plain',...",NaN,NaN,Mitch McConnell is the Republican leader in th...
3,27a3bb38-f74a-11e4-9ef4-1bb7ce3b3fb7,https://www.washingtonpost.com/local/local-dig...,"Local digest: May 11, 2015",— Michael Smith; — Peter Hermann,2015-05-10,article,The Washington Post,"[{'content': 'Local', 'mime': 'text/plain', 't...",NaN,NaN,A man was killed in a shooting early Sunday mo...
4,24e0ddf8-1b29-11e3-82ef-a059e54c49d0,https://www.washingtonpost.com/local/man-sente...,Man sentenced to 27 years in Carl Diener murder,Rachel Weiner,2013-09-12,article,The Washington Post,"[{'content': 'Local', 'mime': 'text/plain', 't...",NaN,NaN,"After delivering a poetic, Scripture-laced den..."


In [218]:
df['title'] = df['title'].replace('', None)
df['title'] = df['title'].replace(' ', None)
df['title'] = df['title'].replace('None', None)
df['title'] = df['title'].replace(np.nan, None)
df['title'] = df.title.replace(r'^\s*$', None, regex=True)

df.dropna(subset=["title"], inplace=True)

def combineText(x):
    try:
        mergedText = f'{x.title} {x.text}'
        return mergedText
    except:
        return None

# write Title and Text in merged_total_text
df['merged_total_text'] = df.parallel_apply(combineText, axis=1)

# | Named Entity Recognition w/ spaCy

In [219]:
def createEntitys(x):
    import spacy
    import en_core_web_trf

    # TRF Orininal / Md schneller
    nlp = spacy.load("en_core_web_md")
    # JULIA nlp = spacy.load("en_core_web_trf")
    
    totalText = x['merged_total_text']
    try:
        entitys = {}
        for word in nlp(totalText).ents:
            entitys[str(word)] = word.label_
        return entitys
    except:
        return None

df['entitys'] = df.parallel_apply(createEntitys, axis=1)

In [220]:
#!pip install gender_guesser  
df['author'] = df['author'].replace('', None)
df['author'] = df['author'].replace(' ', None)
df['author'] = df['author'].replace('None', None)
df['author'] = df['author'].replace(np.nan, None)
# df['author'] = df.author.replace(r'^\s*$', None, regex=True)

def authorGender(x):
    try:
        import gender_guesser.detector as gender
        gd = gender.Detector()
        author = x.author
        author = author.split(' ')[0]
        gender = gd.get_gender(author)
        return gender
    except:
        return None

df['author_gender'] = df.parallel_apply(authorGender, axis=1)

In [221]:
def findCategory(x):
    try:
        articleUrl = x.article_url

        if articleUrl[:5] == 'https':
            category = articleUrl.split('/')[3]
        elif articleUrl[:1] == '/':
            category = articleUrl.split('/')[1]
        else:
            category = 'other'
        return category
    except:
        return None

# find category from article_url
df['category'] = df.parallel_apply(findCategory, axis=1)

In [222]:
def communion_context(x):
    try:
        file = 'a_communion.txt'
        with open(file, "r") as tf:
            lines = tf.read().split('\n')
        print(lines)
        import re
        counter = 0
        for line in lines:
            count = sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(line), x.title))
            counter += count
        return counter
    except:
        return None

df['communion_context'] = df.parallel_apply(communion_context, axis=1)

In [223]:
def agency_context(x):
    try:
        file = 'a_agency.txt'
        with open(file, "r") as tf:
            lines = tf.read().split('\n')
        print(lines)
        import re
        counter = 0
        for line in lines:
            count = sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(line), x.title))
            counter += count
        return counter
    except:
        return None

df['agency_context'] = df.parallel_apply(agency_context, axis=1)

In [224]:
def getTitleLenght(x):
    try:
        return len(x.title)
    except:
        return None

# write length of title
df['len_title'] = df.parallel_apply(getTitleLenght, axis=1)

In [225]:
def getTitleWordLenght(x):
    try:
        title = x.title
        import spacy
        nlp = spacy.load('en_core_web_sm')
        title = nlp(title)
        tokenizerLength = len(title)
        return tokenizerLength
    except:
        return None

# write number of title words
df['word_title'] = df.parallel_apply(getTitleWordLenght, axis=1)

In [226]:
def TitleVaderScore(x):
    try:
        from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
        analyzer = SentimentIntensityAnalyzer()
        title = x.title
        vader_score = analyzer.polarity_scores(title)['compound']
        if vader_score >= 0.05 : 
            return("Positive") 
        elif vader_score <= - 0.05 : 
            return("Negative")  
        else : 
            return("Neutral")
    except:
        return None

# find category from article_url
df['compund'] = df.parallel_apply(TitleVaderScore, axis=1)

In [228]:
# Speichern des aktuellen Stands

aktuellPickle = 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\aktuell.pickle'
if os.path.exists(aktuellPickle):
    os.remove(aktuellPickle)
df.to_pickle(aktuellPickle)

In [229]:
df

,id,article_url,title,author,published_date,type,source,content,publish_date,orig-id,text,merged_total_text,entitys,author_gender,category,communion_context,agency_context,len_title,word_title,compund
0,ABM35HDPIII6RP4GUI2RWXWOTE,/opinions/the-hollow-compassion-of-trump-wary-...,Trump’s America goes full Charles Dickens,Dana Milbank,NaN,NaN,NaN,"[{'content': 'Opinions', 'mime': 'text/plain',...",NaN,NaN,Donald Trumps America has taken on a Dickensia...,Trump’s America goes full Charles Dickens Dona...,"{'Trump’s America': 'ORG', 'Charles Dickens Do...",female,opinions,0,0,41,7,Neutral
1,7f4cc4bc-351a-11e4-9e92-0899b306bbea,https://www.washingtonpost.com/local/u-md-bask...,U-Md. basketball arena is transformed into mak...,Karen Chen,2014-09-06,article,The Washington Post,"[{'content': 'Local', 'mime': 'text/plain', 't...",NaN,NaN,Outside the University of Marylands basketball...,U-Md. basketball arena is transformed into mak...,"{'U-Md.': 'ORG', 'Saturday Outside the Univers...",female,local,0,0,83,13,Neutral
2,aa34106e-bc85-11e1-8867-ecf6cb7935ef,https://www.washingtonpost.com/opinions/mitch-...,Mitch McConnell: How political disclosure coul...,Mitch McConnell,2012-06-23,article,The Washington Post,"[{'content': 'Opinions', 'mime': 'text/plain',...",NaN,NaN,Mitch McConnell is the Republican leader in th...,Mitch McConnell: How political disclosure coul...,"{'Mitch McConnell': 'PERSON', 'Republican': 'N...",male,opinions,0,0,68,10,Positive
3,27a3bb38-f74a-11e4-9ef4-1bb7ce3b3fb7,https://www.washingtonpost.com/local/local-dig...,"Local digest: May 11, 2015",— Michael Smith; — Peter Hermann,2015-05-10,article,The Washington Post,"[{'content': 'Local', 'mime': 'text/plain', 't...",NaN,NaN,A man was killed in a shooting early Sunday mo...,"Local digest: May 11, 2015 A man was killed in...","{'May 11, 2015': 'DATE', 'early Sunday morning...",unknown,local,0,0,26,7,Neutral
4,24e0ddf8-1b29-11e3-82ef-a059e54c49d0,https://www.washingtonpost.com/local/man-sente...,Man sentenced to 27 years in Carl Diener murder,Rachel Weiner,2013-09-12,article,The Washington Post,"[{'content': 'Local', 'mime': 'text/plain', 't...",NaN,NaN,"After delivering a poetic, Scripture-laced den...",Man sentenced to 27 years in Carl Diener murde...,"{'27 years': 'DATE', 'Carl Diener': 'PERSON', ...",female,local,0,0,47,9,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,8caa65930287717ff4e6b8aacd58cc6d,https://www.washingtonpost.com/news/in-sight/w...,How palm oil cultivation in Borneo is threaten...,Nicole Crowder,2015-12-01,blog,The Washington Post,"[{'content': 'In Sight', 'mime': 'text/plain',...",NaN,NaN,Ahead of the Paris Sustainable Innovation Foru...,How palm oil cultivation in Borneo is threaten...,"{'Borneo': 'GPE', 'this month': 'DATE', 'Matti...",female,news,0,0,74,11,Negative
296,f9037ca38dd6f3f594fb8d77d6cbc1fa,https://www.washingtonpost.com/news/capital-we...,"Significant snowstorm Tuesday, with widespread...",Wes Junker and Jason Samenow,2014-01-20,blog,The Washington Post,"[{'content': 'Capital Weather Gang', 'mime': '...",NaN,NaN,* Winter storm warning Tuesday for entire regi...,"Significant snowstorm Tuesday, with widespread...","{'snowstorm Tuesday': 'DATE', '4″': 'CARDINAL'...",male,news,0,0,68,10,Positive
297,45e8e043fe562887c16737d4c5f79fea,https://www.washingtonpost.com/news/early-lead...,"Patriots’ Vince Wilfork gets cut, keeps it classy",Matt Bonesteel,2015-03-05,blog,The Washington Post,"[{'content': 'Early Lead', 'mime': 'text/plain...",NaN,NaN,The Patriots apparently are moving forward wit...,"Patriots’ Vince Wilfork gets cut, keeps it cla...","{'Patriots': 'ORG', 'Vince Wilfork': 'PERSON',...",male,news,0,0,49,10,Positive
298,60a8758a-827d-11e1-ba3d-cb532dc48601,https://www.washingtonpost.com/local/traffican...,"Council, Leggett spar over Wheaton’s future",Lori Aratani,2012-04-10,article,The Washington Post,"[{'content': 'Transportation', 'mime': 'text/p...",NaN,NaN,The Montgomery County C

---
---
---
---
---
---

# UNFERTIGE FUNKTIONEN

In [84]:
# Laden des aktuellen Stands

aktuellPickle = 'C:\\Users\\Felix\\github\\privat\\dis25_abgabe\\data\\wpdata\\data\\aktuell.pickle'
df = pd.read_pickle(aktuellPickle)

In [86]:
df.head(5)

,id,article_url,title,author,published_date,type,source,content,publish_date,orig-id,text,merged_total_text,entitys
0,ABM35HDPIII6RP4GUI2RWXWOTE,/opinions/the-hollow-compassion-of-trump-wary-...,Trump’s America goes full Charles Dickens,Dana Milbank,NaN,NaN,NaN,"[{'content': 'Opinions', 'mime': 'text/plain',...",NaN,NaN,Donald Trumps America has taken on a Dickensia...,Trump’s America goes full Charles Dickens Dona...,"{'Trump’s America': 'ORG', 'Charles Dickens Do..."
1,7f4cc4bc-351a-11e4-9e92-0899b306bbea,https://www.washingtonpost.com/local/u-md-bask...,U-Md. basketball arena is transformed into mak...,Karen Chen,2014-09-06,article,The Washington Post,"[{'content': 'Local', 'mime': 'text/plain', 't...",NaN,NaN,Outside the University of Marylands basketball...,U-Md. basketball arena is transformed into mak...,"{'U-Md.': 'ORG', 'Saturday Outside the Univers..."
2,aa34106e-bc85-11e1-8867-ecf6cb7935ef,https://www.washingtonpost.com/opinions/mitch-...,Mitch McConnell: How political disclosure coul...,Mitch McConnell,2012-06-23,article,The Washington Post,"[{'content': 'Opinions', 'mime': 'text/plain',...",NaN,NaN,Mitch McConnell is the Republican leader in th...,Mitch McConnell: How political disclosure coul...,"{'Mitch McConnell': 'PERSON', 'Republican': 'N..."
3,27a3bb38-f74a-11e4-9ef4-1bb7ce3b3fb7,https://www.washingtonpost.com/local/local-dig...,"Local digest: May 11, 2015",— Michael Smith; — Peter Hermann,2015-05-10,article,The Washington Post,"[{'content': 'Local', 'mime': 'text/plain', 't...",NaN,NaN,A man was killed in a shooting early Sunday mo...,"Local digest: May 11, 2015 A man was killed in...","{'May 11, 2015': 'DATE', 'early Sunday morning..."
4,24e0ddf8-1b29-11e3-82ef-a059e54c49d0,https://www.washingtonpost.com/local/man-sente...,Man sentenced to 27 years in Carl Diener murder,Rachel Weiner,2013-09-12,article,The Washington Post,"[{'content': 'Local', 'mime': 'text/plain', 't...",NaN,NaN,"After delivering a poetic, Scripture-laced den...",Man sentenced to 27 years in Carl Diener murde...,"{'27 years': 'DATE', 'Carl Diener': 'PERSON', ..."


In [87]:
# id
# article_url
# title
# author
# published_date
# type
# source
# content
# publish_date
# orig-id
# text
# merged_total_text
# entitys

# df[df['publish_date'].isnull()]

In [89]:
# NER 'PERSON' STARTS HERE
#df = (df.loc[df['entity_type'].isin(['PERSON'])])

In [90]:
#df["entity"] = df['entity'].str.replace('[\[\]\"\'\d\,\<\/]','')

In [91]:
# Replace artist names (Sting, Bono, Cher etc.) with name according to wikipedia
def replaceEntityNames(x):
    try:
        entitys = x.entitys
        artists = {
            'JFK': 'John Fitzgerald Kennedy',
            'FDR': 'Franklin Delano Roosevelt',
            'Sting': 'Gordon Matthew Sumner',
            'Bono': 'Paul David Hewson',
            'Cher': 'Cherilyn Sarkisian',
            'Madonna': 'Madonna Louise Ciccone',
            'Adele': 'Adele Laurie Adkins',
            'Eminem': 'Marshall Bruce Mathers',
            'Beyonce': 'Beyoncé Knowles-Carter',
            'Blaine Friedlander': 'Blaine P. Friedlander Jr.'
        }

        for entityKey, entityValue in entitys.items():
            if entityValue == 'PERSON':
                for artAbkzg, artName in artists.items():
                    if entityKey == artAbkzg:
                        entitys[artName]=entityValue
                        del entitys[entityKey]  
    
        return entitys
    except:
        return x.entitys
# !!! NOCH FEHLERHAFT
#df['entitys'] = df.parallel_apply(replaceEntityNames, axis=1)

In [ ]:
df = df[df['author'] != df['entity']] # if author is in col entity -> remove, we only want "Person" from within the article
# To do: Blaine P. Friedlander Jr. vs. Blaine Friedlander 

## Split first, middle and last name in different cols

In [55]:
# Write first, middle and last name in different cols
# to do: split middle and last name does not work
def split_name(df, var):
    sub_df = df[var].str.split('\\s+', expand=True)
    result = []

    for _, row in sub_df.iterrows():
        info = {'first_name': '', 'middle_name': '', 'last_name': ''}
        n = row.count()

        if n == 0:
            pass
        elif n == 1:
            info['last_name'] = row.iloc[0]
        elif n == 2:
            info['first_name'], info['last_name'] = row.iloc[:2]
        else:
            info['first_name'] = row.iloc[0]
            info['last_name'] = row.iloc[-1]
            info['middle_name'] = ' '.join([(string or '') for string in row.iloc[1:-1]])
        result.append(info)
    return pd.DataFrame(result, index=df.index)

df_names = split_name(df, 'entitys')
df = df.join(df_names)

## | Enrich first name if only last name is mentioned

In [ ]:
names = df.groupby('id')['entity'].apply(lambda x: list(np.unique(x)))
# To do: if only one token ("Obama") drop or don't write to list

In [ ]:
# function for enriching first name if only last name is mentioned
from tqdm import tqdm
tqdm.pandas()
def enrich_firstname(row):
    
    entity = row['entity']#.copy()
    firstname = row['first_name']#.copy()
    lastname = row['last_name']#.copy()
    
    #if len(firstname) == 0:
        # look in list per id for key and write value?
    # elif 
    # bestehende Vornamen nicht überschreiben
    #else:
    #    row['first_name'] = 'unknown'
    return row

df_test = df_test.progress_apply(enrich_firstname, axis=1)


# | Gender Guesser

In [ ]:
# !pip install gender_guesser    
import gender_guesser.detector as gender
gd = gender.Detector()
df['gender_guesser'] = df['first_name'].apply(str.capitalize).map(lambda x: gd.get_gender(x))

> __unknown__ (name not found), __andy__ (androgynous), __male__, __female__, __mostly_male__, or __mostly_female__. The difference between andy and unknown is that the former is found to have the same probability to be male than to be female, while the later means that the name wasn’t found in the database.

In [ ]:
df.gender_guesser.value_counts()

In [ ]:
df.loc[df['gender_guesser'] == 'unknown'][:40]

In [ ]:
# df.first_name.value_counts()[:40]

In [ ]:
# to do:
# Namen cleanen
# weitere lib für gender identification einbauen
# function schreiben: wenn nur Nachname genannt, schaue in Liste/anderer Zeile pro Artikel, ob Name schon genannt, dann Vorname auffüllen
# oder
# wenn Obama, Santorum, Gingrich, etc. dann aus erstellter Liste oder via Wikidata mit Vornamen auffüllen
# # dominique rodgers cromartie etc. (famous people)  -> Wikidata? 

---
# | Group per article and get share of gender per article

In [ ]:
# next

# | Wikidata Query

In [ ]:
# oder das https://stackoverflow.com/questions/51419785/extract-data-from-wikidata-in-python

import requests

sparql_query = """
        prefix schema: <http://schema.org/>
        SELECT ?item ?occupation ?genderLabel ?bdayLabel
        WHERE {
            <https://en.wikipedia.org/wiki/Angela_Merkel> schema:about ?item .
            ?item wdt:P21 ?gender .
            SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        }
    """

In [ ]:
url = 'https://query.wikidata.org/sparql'

# sleep(2)
r = requests.get(url, params={'format': 'json', 'query': sparql_query})

In [ ]:
url = 'https://query.wikidata.org/sparql'

r = requests.get(url, params={'format': 'json', 'query': sparql_query})
data = r.json()

print(data['results']['bindings'])